In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
np.random.seed(123)

get company data.
filter by type of company:

* SOCIETA' DI CAPITALE|SU|SOCIETA' A RESPONSABILITA' LIMITATA CON UNICO SOCIO
* SOCIETA' DI CAPITALE|SR|SOCIETA' A RESPONSABILITA' LIMITATA
* SOCIETA' DI CAPITALE|SP|SOCIETA' PER AZIONI
* SOCIETA' DI CAPITALE|SD|SOCIETA' EUROPEA
* SOCIETA' DI CAPITALE|RS|SOCIETA' A RESPONSABILITA' LIMITATA SEMPLIFICATA
* SOCIETA' DI CAPITALE|RR|SOCIETA' A RESPONSABILITA' LIMITATA A CAPITALE RIDOTTO
* SOCIETA' DI CAPITALE|AU|SOCIETA'  PER AZIONI CON SOCIO UNICO
* SOCIETA' DI CAPITALE|AA|SOCIETA' IN ACCOMANDITA PER AZIONI

In [2]:
# Data Acquisition
filename = r'../../_DataScience/____PHD_2021/_data/tidy/cmp.csv'
# cols_to_use = ['idCompany', 'name', 'cf', 'prov', 'sede_ul', 'ng2', 'stato_impresa',
#        'addetti_aaaa', 'addetti_indip', 'addetti_dip', 'capitale',
#        'capitale_valuta', 'imp_sedi_ee', 'imp_eefvg', 'is.sme', 'is.startup',
#        'is.fem', 'is.young', 'is.fore', 'yearsInBusiness']
cols_to_use = [ 'cf', 'prov', 'ng2', 'stato_impresa', 'yearsInBusiness']
companies = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
companies['yearsInBusiness'] = companies['yearsInBusiness'].astype(float).round(1)
company_types = ['SU','SR', 'SP','SD','RS','RR','AU','AA']
companies = companies[companies.prov.isin(['TS','GO','UD','PN'])]
companies = companies[companies.ng2.isin(company_types)]
companies = companies[companies.stato_impresa.isin(['ATTIVA'])]
companies.shape
cols_to_keep = ['cf','yearsInBusiness']

companies.drop_duplicates(inplace=True)

companies = companies[ cols_to_keep ]


In [3]:
filename = r'../../_DataScience/____PHD_2021/_data/tidy/bsd.csv'
# cols_to_use = ['cf', 'prov', 'year', 'totAssets', 'totIntang', 'accounts', 'totEquity',
#        'debts', 'prod', 'revenues', 'personnel', 'valCost', 'ammort',
#        'profLoss', 'valAdded', 'deprec', 'noi']
cols_to_use = ['cf', 'year', 'totEquity', 'totAssets',  'noi', 
               'revenues', 'profLoss', 'valAdded', 'personnel', 
               'debts', 'deprec', 'totIntang']
bsd = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
bsd = bsd[bsd.year == '2019']
bsd['totEquity'] = bsd['totEquity'].astype(float)
bsd['totAssets'] = bsd['totAssets'].astype(float)
bsd['noi'] = bsd['noi'].astype(float)
bsd['revenues'] = bsd['revenues'].astype(float)
bsd['profLoss'] = bsd['profLoss'].astype(float)
bsd['valAdded'] = bsd['valAdded'].astype(float)
bsd['personnel'] = bsd['valAdded'].astype(float)
bsd['debts'] = bsd['debts'].astype(float)
bsd['deprec'] = bsd['deprec'].astype(float)
bsd['totIntang'] = bsd['totIntang'].astype(float)


bsd['rAssets'] = bsd.totAssets/bsd.totEquity
bsd['rNOI'] =    bsd.noi/bsd.totEquity
bsd['rIntang'] = bsd.totIntang/bsd.totEquity


cols_to_use = ['cf', 'totEquity', 'totAssets',  'noi', 
               'revenues', 'profLoss', 'valAdded', 'personnel', 
               'debts', 'deprec', 'totIntang', 'rAssets', 'rNOI', 'rIntang']
bsd = bsd[ cols_to_use]

bsd.head(5)

,cf,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang
2,00002070324,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202
9,00007080369,47371267.0,79743062.0,5431808.0,45248573.0,4340839.0,11430290.0,11430290.0,20448910.0,1467182.0,2093012.0,1.683364,0.114665,0.044183
16,00007470933,84661.0,2244578.0,45825.0,3578453.0,-46369.0,1539618.0,1539618.0,1766385.0,302860.0,351982.0,26.512538,0.541276,4.157546
23,00009840315,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756
30,00012670303,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106


In [4]:
companies = companies.merge(bsd, on='cf')

companies.head(5)

,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202
1,00007470933,59.2,84661.0,2244578.0,45825.0,3578453.0,-46369.0,1539618.0,1539618.0,1766385.0,302860.0,351982.0,26.512538,0.541276,4.157546
2,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756
3,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106
4,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,120729.0,194411.0,6.935805,0.115856,0.231687


In [5]:
cols_to_use = ['cf', 'yearsInBusiness',
               'totEquity', 'totAssets',  'noi', 
               'revenues', 'profLoss', 'valAdded', 'personnel', 
               'debts', 'deprec', 'totIntang', 
               'rAssets', 'rNOI', 'rIntang']
companies=companies[cols_to_use]
companies.head(5)

,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202
1,00007470933,59.2,84661.0,2244578.0,45825.0,3578453.0,-46369.0,1539618.0,1539618.0,1766385.0,302860.0,351982.0,26.512538,0.541276,4.157546
2,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756
3,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106
4,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,120729.0,194411.0,6.935805,0.115856,0.231687


In [6]:
filename = r'../../_DataScience/____PHD_2021/_data/tidy/rating.csv'
#cols_to_use = ['cf', 'final_rank', 'evaluation_date', 'is_consolidated', 'rating010','year']
cols_to_use = ['cf', 'rating010','year']
rating = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
rating = rating[rating.year == '2019']
rating['rating010'] = rating['rating010'].astype(float)
rating.head(5)

,cf,rating010,year
2,00008980328,1.0,2019
6,00019410307,1.0,2019
10,00037070323,1.0,2019
14,00039970314,1.0,2019
18,00041170317,1.0,2019


In [7]:
companies = companies.merge(rating, on='cf')
companies.head(5)
 

,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang,rating010,year
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202,9.0,2019
1,00007470933,59.2,84661.0,2244578.0,45825.0,3578453.0,-46369.0,1539618.0,1539618.0,1766385.0,302860.0,351982.0,26.512538,0.541276,4.157546,5.0,2019
2,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756,9.0,2019
3,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106,6.0,2019
4,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,120729.0,194411.0,6.935805,0.115856,0.231687,2.0,2019


In [8]:
cols_to_use = ['cf', 'yearsInBusiness', 'totEquity', 'totAssets',  'noi', 
               'revenues', 'profLoss', 'valAdded', 'personnel', 
               'debts', 'deprec', 'totIntang', 
               'rAssets', 'rNOI', 'rIntang', 'rating010']
companies=companies[cols_to_use]
companies.head(5)

,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang,rating010
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202,9.0
1,00007470933,59.2,84661.0,2244578.0,45825.0,3578453.0,-46369.0,1539618.0,1539618.0,1766385.0,302860.0,351982.0,26.512538,0.541276,4.157546,5.0
2,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756,9.0
3,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106,6.0
4,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,120729.0,194411.0,6.935805,0.115856,0.231687,2.0


In [9]:
filename = r'../../_DataScience/____PHD_2021/_data/tidy/nace.csv'
#cols_to_use = ['cf', 'idCompany', 'id_localiz', 'loc_n', 'code_type', 'division','code']
cols_to_use = ['cf', 'loc_n', 'code_type', 'division']
nace = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
nace = nace[ nace.code_type == "P"]
nace = nace[ nace.loc_n == "0"] 
nace.drop_duplicates(subset=['cf'], keep='first', inplace=True, ignore_index=True)
cols_to_use = ['cf', 'division']
nace=nace[cols_to_use]
nace.columns = ['cf', 'NACE_division']
nace.head(5)


,cf,NACE_division
0,00002070324,52
1,00003930328,47
2,00006350938,45
3,00006500938,16
4,00007080369,46


In [10]:
nace.shape

(36468, 2)

In [11]:
companies = companies.merge(nace, on='cf')
companies.head(5)


,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang,rating010,NACE_division
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,18574.0,2515.0,3.112637,1.003312,0.003202,9.0,52
1,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,91440.0,93363.0,1.380727,0.197934,0.025756,9.0,47
2,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,204422.0,12857.0,1.328204,0.002143,0.003106,6.0,68
3,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,120729.0,194411.0,6.935805,0.115856,0.231687,2.0,28
4,00025330937,56.6,2171727.0,2244020.0,-34932.0,408995.0,-19027.0,35568.0,35568.0,55927.0,28214.0,10.0,1.033288,-0.016085,0.000005,7.0,17


In [12]:
filename = r'../../_DataScience/____PHD_2021/_data/tidy/empl_stock.csv'
#cols_to_use = ['cf', 'name', 'rea', 'prov', 'StockProv', 'StockAll', 'date_stock']
cols_to_use = ['cf', 'prov','StockProv']
emp_stock = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
emp_stock = emp_stock[emp_stock.prov.isin(['TS','GO','UD','PN'])]
emp_stock.drop_duplicates(inplace=True)
emp_stock['staffCount'] = emp_stock['StockProv'].astype(int)
emp_stock.head(25)
emp_stock.shape
# emp_stock.drop_duplicates(inplace=True)

# companies = companies.merge(emp_stock, on='cf')
# companies.head(20)
 

(32334, 4)

In [13]:
emp_stock

,cf,prov,StockProv,staffCount
0,02391740301,UD,14,14
1,02376010308,UD,0,0
2,02375800303,UD,17,17
3,02385450305,UD,0,0
4,02621450283,UD,1,1
...,...,...,...,...
36077,04196790283,PN,0,0
36078,05881510829,TS,0,0
36080,02944030309,GO,0,0
36081,03211120104,TS,0,0


In [14]:
emp_stock = emp_stock.pivot_table(values = 'staffCount', index = 'cf', aggfunc = np.sum)
emp_stock

,staffCount
cf,
00002070324,16
00007080369,2
00007470933,24
00008980328,0
00009840315,29
...,...
94073780309,5
94089650306,1
94104320307,2


In [15]:
companies = companies.merge(emp_stock, on='cf', how = 'left').fillna(0)


In [16]:
filename = r'../../_DataScience/____PHD_2021/_data/tidy/empl_flows.csv'
cols_to_use = ['cf', 'year', 'turnover','balance']
empl_flows = pd.read_csv(filename, dtype=str, usecols=cols_to_use)
empl_flows = empl_flows[ empl_flows.year == '2019']
empl_flows['cf'] = empl_flows['cf'].str.strip()
empl_flows.drop_duplicates(inplace=True)
empl_flows.columns = ['cf', 'year', 'staff_turnover', 'staff_variation']

In [17]:
companies = companies.merge(empl_flows, on='cf', how = 'left').fillna(0)


In [18]:
companies.head(25)

,cf,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,...,totIntang,rAssets,rNOI,rIntang,rating010,NACE_division,staffCount,year,staff_turnover,staff_variation
0,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,...,2515.0,3.112637,1.003312,0.003202,9.0,52,16.0,2019,3,1
1,00002070324,53.0,785406.0,2444684.0,788007.0,3166261.0,761405.0,1507359.0,1507359.0,1400533.0,...,2515.0,3.112637,1.003312,0.003202,9.0,52,16.0,2019,4,0
2,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,...,93363.0,1.380727,0.197934,0.025756,9.0,47,29.0,2019,13,1
3,00009840315,58.5,3624935.0,5005047.0,717498.0,7013487.0,545112.0,1600331.0,1600331.0,862050.0,...,93363.0,1.380727,0.197934,0.025756,9.0,47,29.0,2019,11,1
4,00012670303,51.1,4138797.0,5497166.0,8869.0,629928.0,23037.0,251424.0,251424.0,1094800.0,...,12857.0,1.328204,0.002143,0.003106,6.0,68,1.0,2019,3,1
5,00018160309,54.6,839111.0,5819910.0,97216.0,9719890.0,46150.0,1194294.0,1194294.0,4388746.0,...,194411.0,6.935805,0.115856,0.231687,2.0,28,25.0,2019,4,0
6,00025330937,56.6,2171727.0,2244020.0,-34932.0,408995.0,-19027.0,35568.0,35568.0,55927.0,...,10.0,1.033288,-0.016085,0.000005,7.0,17,1.0,0,0,0
7,00027730316,55.4,583739.0,3285050.0,351509.0,3882106.0,224386.0,1841987.0,1841987.0,1679403.0,...,30823.0,5.627601,0.602168,0.052803,5.0,25,34.0,2019,35,7
8,00027730316,55.4,583739.0,3285050.0,351509.0,3882106.0,224386.0,1841987.0,1841987.0,1679403.0,...,30823.0,5.627601,0.602168,0.052803,5.0,25,34.0,2019,1,1
9,00030810311,47.0,1166668.0,4800030.0,-91294.0,1142508.0,-147517.0,51755.0,51755.0,2652305.0,...,6696.0,4.114307,-0.078252,0.005739,4.0,47,6.0,0,0,0


In [19]:
coords = pd.read_csv(r'./maps/FVG/companies.csv', dtype='str')
coords.columns = ['ind', 'cf', 'company','unit', 'lat', 'lon']
coords['lat'] = coords['lat'].astype(float)
coords['lon'] = coords['lon'].astype(float)
coords = coords[ coords.unit == 'SEDE']
coords = coords[ [ 'cf',  'lat', 'lon'] ]
coords.shape 

(16624, 3)

In [20]:
companies = companies.merge(coords, on='cf')


In [21]:
#shuffle
companies = companies.sample(frac=1) 

In [22]:
companies.reset_index(inplace=True)

In [23]:
companies.columns = ['id', 'cf', 'yearsInBusiness', 'totEquity', 'totAssets', 'noi',
       'revenues', 'profLoss', 'valAdded', 'personnel', 'debts', 'deprec',
       'totIntang', 'rAssets', 'rNOI', 'rIntang', 'rating010', 'NACE_division',
       'staff_count', 'year', 'staff_turnover', 'staff_variation', 'lat',
       'lon']

In [24]:
companies.sort_values(by='id', inplace=True)

In [29]:
companies['id'] = companies['id'].astype(int)

In [30]:
# save index-cf
tmp = companies [ ['id', 'cf'] ]
tmp.to_csv(r'./data/id_cf.csv', index=False)

In [31]:
# save data without cf
cols_sorted = ['id', 'lon', 'lat',
               'yearsInBusiness', 'NACE_division',
               'totEquity', 'totAssets', 
               'noi', 'revenues', 'profLoss', 'valAdded', 
               'personnel', 'debts', 'deprec', 'totIntang', 
               'rAssets', 'rNOI', 'rIntang', 'rating010', 
               'staff_count', 'staff_turnover', 'staff_variation' ]
tmp = companies [ cols_sorted ] 
tmp.to_csv(r'./data/data.csv', index=False)

In [32]:
companies.describe()

,id,yearsInBusiness,totEquity,totAssets,noi,revenues,profLoss,valAdded,personnel,debts,deprec,totIntang,rAssets,rNOI,rIntang,rating010,staff_count,lat,lon
count,11278.000000,11278.000000,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,1.127800e+04,11278.000000,11278.000000,11278.000000,11278.000000
mean,5638.500000,22.412777,3.323100e+06,8.740546e+06,3.850058e+05,5.898749e+06,2.592852e+05,1.808570e+06,1.808570e+06,3.693817e+06,3.122691e+05,5.838982e+05,inf,-inf,inf,6.063575,21.863540,45.944201,13.172583
std,3255.822502,15.422716,3.862974e+07,1.311247e+08,6.941347e+06,4.960441e+07,4.234447e+06,2.124023e+07,2.124023e+07,7.881237e+07,3.547082e+06,1.404394e+07,NaN,NaN,NaN,1.700910,103.933789,0.187140,0.393347
min,0.000000,0.800000,-2.297749e+07,-4.323000e+03,-3.561701e+07,-3.000000e+00,-4.499855e+07,-2.676867e+07,-2.676867e+07,-4.017200e+04,-2.399300e+04,-9.001000e+03,-1.754748e+03,-inf,-1.187091e+02,0.000000,0.000000,45.586202,12.331641
25%,2819.250000,9.900000,4.869150e+04,2.804252e+05,3.752000e+03,1.758578e+05,2.132500e+02,4.634350e+04,4.634350e+04,8.808875e+04,4.771250e+03,0.000000e+00,1.756054e+00,3.469116e-02,0.000000e+00,5.000000,1.000000,45.826110,12.826040
50%,5638.500000,19.400000,2.056635e+05,8.537460e+05,2.769600e+04,6.627885e+05,1.194150e+04,2.112435e+05,2.112435e+05,2.903165e+05,2.054000e+04,2.596500e+03,3.069007e+00,1.657233e-01,7.442427e-03,6.000000,6.000000,45.957822,13.222327
75%,8457.750000,31.900000,8.736510e+05,2.762645e+06,1.140268e+05,2.240834e+06,6.949525e+04,7.377120e+05,7.377120e+05,9.996842e+05,7.643975e+04,2.948600e+04,6.434973e+00,4.111435e-01,9.075414e-02,8.000000,16.000000,46.062487,13.443402
max,11277.000000,109.200000,1.629648e+09,6.604382e+09,3.467160e+08,1.979299e+09,1.538490e+08,1.028790e+09,1.028790e+09,4.111092e+09,1.493900e+08,6.608760e+08,inf,2.146667e+02,inf,10.000000,3775.000000,46.603854,13.863699
